In [14]:
import os
import cv2
import shutil
import albumentations as A
import random
from glob import glob
from ultralytics import YOLO
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

In [15]:
class_names = {
    'box1',
    'box2',
    'box3',
    'reject',
}

In [16]:
os.makedirs('YOLO/original_img_lab/images', exist_ok=True)

In [10]:
os.makedirs('YOLO/original_img_lab/labels', exist_ok=True)

In [18]:
transform = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.ShiftScaleRotate(
            shift_limit=0.02,
            scale_limit=0.1,
            rotate_limit=5,
            border_mode=cv2.BORDER_CONSTANT,
            value=(114, 114, 114),
            p=0.5,
        ),
    ],
    bbox_params=A.BboxParams(
        format="yolo",
        label_fields=["class_labels"],
        min_visibility=0.3,
        check_each_transform=True,
    ),
)

os.makedirs('YOLO/img_lab/images', exist_ok=True)
os.makedirs('YOLO/img_lab/labels', exist_ok=True)

for img_name in os.listdir('YOLO/original_img_lab/images'):
    if not img_name.lower().endswith('.png'):
        continue
    base, _ = os.path.splitext(img_name)
    img = cv2.imread(f'YOLO/original_img_lab/images/{img_name}')
    if img is None:
        continue
    label_path = f'YOLO/original_img_lab/labels/{base}.txt'
    with open(label_path) as f:
        labels = [list(map(float, line.split())) for line in f if len(line.split()) == 5]
    bboxes = [x[1:] for x in labels]
    classes = [int(x[0]) for x in labels]
    for i in range(100):
        transformed = transform(image=img, bboxes=bboxes, class_labels=classes)
        if transformed['bboxes']:
            cv2.imwrite(f'YOLO/img_lab/images/{base}_{i}.jpg', transformed['image'])
            with open(f'YOLO/img_lab/labels/{base}_{i}.txt', 'w') as f:
                for (x, y, w, h), c in zip(transformed['bboxes'], transformed['class_labels']):
                    f.write(f"{c} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

/tmp/ipykernel_26473/4149926687.py:5: UserWarning: Argument(s) 'value' are not valid for transform ShiftScaleRotate
  A.ShiftScaleRotate(


In [19]:
root = Path('YOLO/img_lab')
splits = {'train': 0.8, 'test': 0.1, 'valid': 0.1}

for split in splits:
    (root / split / 'images').mkdir(parents=True, exist_ok=True)
    (root / split / 'labels').mkdir(parents=True, exist_ok=True)

src_images = root / 'images'
src_labels = root / 'labels'

files = [f for f in src_images.iterdir() if f.suffix.lower() in ('.jpg', '.jpeg', '.png')]
random.shuffle(files)

n = len(files)
n_train = int(n * splits['train'])
n_test  = int(n * splits['test'])

train_files = files[:n_train]
test_files  = files[n_train:n_train + n_test]
valid_files = files[n_train + n_test:]

for split, flist in (('train', train_files), ('test', test_files), ('valid', valid_files)):
    for img in flist:
        (root / split / 'images' / img.name).write_bytes(img.read_bytes())
        label_src = src_labels / f'{img.stem}.txt'
        if label_src.exists():
            (root / split / 'labels' / label_src.name).write_bytes(label_src.read_bytes())

In [20]:
model = YOLO("yolo26n.pt")

In [21]:
%%writefile data.yaml
train: YOLO/img_lab/train/images
val: YOLO/img_lab/valid/images
test: YOLO/img_lab/test/images

nc: 4
names: [box1, box2, box3, reject]

Overwriting data.yaml


In [22]:
result = model.train(
    data='data.yaml',
    epochs=3,
    batch=8,
    imgsz=640
)

New https://pypi.org/project/ultralytics/8.4.8 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.4.7 🚀 Python-3.10.12 torch-2.9.1+cu128 CPU (13th Gen Intel Core i5-13400)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize